#1. 모델 이해하기

합성곱 신경망은 출처에 따라서 합성곱 층을 부르는 단위가 조금 다르다.

##1. 첫번째 표기 방법

합성곱(nn.Cov2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고, 맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도로 명명한다.

##2. 두번째 표기 방법

합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 본다.

모델의 아키텍쳐는 총 3개의 층으로 구성된다.

In [1]:
# # 1번 레이어 : 합성곱층(Convolutional layer)
# 합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
# 맥스풀링(kernel_size=2, stride=2))

# # 2번 레이어 : 합성곱층(Convolutional layer)
# 합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
# 맥스풀링(kernel_size=2, stride=2))

# # 3번 레이어 : 전결합층(Fully-Connected layer)
# 특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
# 전결합층(뉴런 10개) + 활성화 함수 Softmax

#2. 모델 구현하기


##1. 필요한 도구 임포트와 입력의 정의

In [2]:
import torch
import torch.nn as nn

임의의 텐서 만들기 / 크기는 1 × 1 × 28 × 28

In [3]:
inputs = torch.Tensor(1,1,28,28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


##2. 합성곱층과 풀링 선언하기

첫번쨰 합성곱 층 구현 / 1채널 짜리의 입력을 받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1

In [4]:
conv1 = nn.Conv2d(1,32,3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


두번째 합성곱 층 구현 / 32채널 짜리를 입력받아서 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1

In [6]:
conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


맥스풀링 구현 / 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당 값으로 지정된다.

In [7]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


##3. 구현체를 연결하여 모델 만들기

입력을 첫번째 합성곱층을 통과시키고 합성곱층을 통과시킨 후의 텐서의 크기

In [12]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


32채널의 28너비 28높이의 텐서가 되었다.  
맥스풀링을 통과한 후의 텐서의 크기

In [13]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


32채널의 14너비 14높이의 텐서가 되었다.   
다시 두번째 합성곱층에 통과시킨후 텐서의 크기

In [14]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


64채널의 14너비 14높이의 텐서  
맥스풀링을 통과한 후의 텐서의 크기

In [15]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


.view()를 사용하여 텐서를 펼침

In [16]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고모두 하나의 차원으로통합된 것을 볼 수 있다. 이제 이에 대해서 전결합층을 통과시키면 출력층 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환

In [17]:
fc = nn.Linear(3136,10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


#3. CNN으로 MNIST 분류하기

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

# if device == 'cuda':
#     torch.cuda.maual_seed_all(777)

AttributeError: ignored

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 351214435.61it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 125659433.43it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 141649422.35it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 25743957.79it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [6]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [7]:
model = CNN().to(device)

In [8]:
#비용함수와 옵티마이저
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [10]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225685298
[Epoch:    2] cost = 0.0629231557
[Epoch:    3] cost = 0.046287328
[Epoch:    4] cost = 0.0375151299
[Epoch:    5] cost = 0.0312200896
[Epoch:    6] cost = 0.0259902645
[Epoch:    7] cost = 0.0217699111
[Epoch:    8] cost = 0.0180980954
[Epoch:    9] cost = 0.0160830449
[Epoch:   10] cost = 0.0130677316
[Epoch:   11] cost = 0.00975070149
[Epoch:   12] cost = 0.00971365348
[Epoch:   13] cost = 0.00819949061
[Epoch:   14] cost = 0.00650474429
[Epoch:   15] cost = 0.00631104643


In [11]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9875999689102173


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
